In [ ]:
import re
import json
import pandas as pd
import matplotlib.pyplot as plt
import os
from datetime import datetime, timedelta

In [ ]:
# TIME-BASED SEARCH

# Define your time range
start_time = datetime(2024, 2, 28, 0, 0)  
end_time = datetime(2024, 2, 28, 23, 0)     

def is_within_range(file_name, start_time, end_time):
    # Extract the date and time from the file name
    try:
        # Split the filename and remove the '.csv' part
        date_str = file_name.split('/')[-1].split('_')[1].rsplit('.', 1)[0]
        file_time = datetime.strptime(date_str, '%Y-%m-%d.%H.%M.%S')
    except ValueError:
        # If conversion fails, return False
        return False

    # Check if the file time is within the range
    return start_time <= file_time <= end_time


# Directory containing the files
directory = "./eval/results/"

file_count = 0
success_count = 0
file_list = []

# Process each file
for file_name in os.listdir(directory):
    if file_name.endswith(".csv") and is_within_range(file_name, start_time, end_time):
        file_list.append(file_name)
        file_count += 1
        file_path = os.path.join(directory, file_name)
        # Process the file
        print(f"Processing {file_path}")
        
        df = pd.read_csv(file_path)
                
        print(f"Starting z-score: {df.iloc[0]['watermark_score']}")
        
        print(f"Total perturbation attemps: {len(df) - 1}")
        
        count = df[df['mutated_text_len'] < 0.95 * df['current_text_len']].shape[0]
        print(f"Short length count: {count}")
        
        quality_preserved = df['quality_preserved'].sum() - 1
        print(f"Quality preserved perturbations: {quality_preserved}")
        
        attack_successful = (not df.iloc[-1]['watermark_detected']) and (df.iloc[-1]['quality_preserved'])
        print(f"Attack successful: {attack_successful}")
        if attack_successful:
            success_count +=1
            
print(f"File count: {file_count}")
print(f"Success count: {success_count}")